## Import necessary libraries

In [1]:
import numpy as np
from numpy import log
import pandas as pd
from scipy.stats import norm
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

# Theory
<p>
    <font size=4>
        The analytical formula of ordinary barrier option is given in Rubinstein's paper:
<p/>
<p style='text-align: center;'>
    <font size=4>
        Parameters: <br>
        $S$: spot price <br>
        $K$: strike <br>
        $r$: risk-free rate <br>
        $q$: dividend yield <br>
        $T$: year-to-maturity <br>
        $\sigma$: annual volatility <br>
        $H$: barrier <br>
        $Re$: rebate <br> <br>
        Variables used in the formula: <br>
        $R:1+r$ <br>
        $Q:1+q$ <br>
        $\mu:ln\frac{R}{Q}-\frac{\sigma^2}{2}$ <br>
        $\lambda:1+\frac{\mu}{\sigma^2}$ <br>
        $x:\frac{ln\frac{S}{K}}{\sigma\sqrt T}+\lambda\sigma\sqrt T$ <br>
        $x_1:\frac{ln\frac{S}{H}}{\sigma\sqrt T}+\lambda\sigma\sqrt T$ <br>
        $y:\frac{ln\frac{H^2}{SK}}{\sigma\sqrt T}+\lambda\sigma\sqrt T$ <br>
        $y_1:\frac{ln\frac{H}{S}}{\sigma\sqrt T}+\lambda\sigma\sqrt T$ <br>
        $a=\frac{\mu}{\sigma^2}$ <br>
        $b=\frac{\sqrt {\mu^2+2(lnR)\sigma^2}}{\sigma^2}$ <br>
        $z:\frac{ln\frac{H}{S}}{\sigma\sqrt T}+b\sigma\sqrt T$ <br><br>
        Formulae for determining value of ordinary barrier option: <br>
        $[1]:\phi SQN(\phi x)-\phi KR^{-T}N(\phi x-\phi\sigma\sqrt T)$ <br>
        $[2]:\phi SQN(\phi x_1)-\phi KR^{-T}N(\phi x-\phi\sigma\sqrt T)$ <br>
        $[3]:\phi SQ^{-T}(\frac{H}{S})^{2\lambda}N(\eta y)-\phi KR^{-T}(\frac{H}{S})^{2\lambda-2}N(\eta y-\eta\sigma\sqrt T)$ <br>
        $[4]:\phi SQ^{-T}(\frac{H}{S})^{2\lambda}N(\eta y_1)-\phi KR^{-T}(\frac{H}{S})^{2\lambda-2}N(\eta y_1-\eta\sigma\sqrt T)$ <br>
        $[5]:ReR^{-T}[N(\eta x_1-\eta\sigma\sqrt T)-(\frac{H}{S})^{2\lambda-2}N(\eta y_1-\eta)]$ <br>
        $[6]:Re[(\frac{H}{S})^{a+b}N(\eta z)+(\frac{H}{S})^{a-b}N(\eta z-2\eta b\sigma\sqrt T)]$ <br>
        where $N(.)$ is the cdf of standard normal <br><br>
        Price of barrier option: <br>
        If up, $\eta=-1$, else $\eta=1$ <br>
        If Call, $\phi=1$, else $\phi=-1$ <br>
        Denote di as down-and-in, do as down-and-out, ui as up-and-in, and uo as up-and-out, we have: <br>
        $C_{di(K\geq H)}=[3]+[5]$ <br>
        $C_{di(K<H)}=[1]-[2]+[4]+[5]$ <br>
        $C_{ui(K\geq H)}=[1]+[5]$ <br>
        $C_{ui(K<H)}=[2]-[3]+[4]+[5]$ <br>
        $P_{di(K\geq H)}=[2]-[3]+[4]+[5]$ <br>
        $P_{di(K<H)}=[1]+[5]$ <br>
        $P_{ui(K\geq H)}=[1]-[2]+[4]+[5]$ <br>
        $P_{ui(K<H)}=[3]+[5]$ <br>
        $C_{do(K\geq H)}=[1]-[3]+[6]$ <br>
        $C_{do(K<H)}=[2]-[4]+[6]$ <br>
        $C_{uo(K\geq H)}=[6]$ <br>
        $C_{uo(K<H)}=[1]-[2]+[3]-[4]+[6]$ <br>
        $P_{do(K\geq H)}=[1]-[2]+[3]-[4]+[6]$ <br>
        $P_{do(K<H)}=[6]$ <br>
        $P_{uo(K\geq H)}=[2]-[4]+[6]$ <br>
        $P_{uo(K<H)}=[1]-[3]+[6]$ <br>
<p/>

In [2]:
def Barrier_Option_Pricer(S, K, r, q, T, sig, H, Re, isCall=True, isUp=True, isOut=True):
    """
    This functions computes the price if barrier option
    input:
    S: Underlying asset spot price
    K: Strike price
    r: risk-free rate (if r = 4%, input 0.04)
    q: continuous dividend yield (if q = 4%, input 0.04) 
    T: Year to maturity
    sig: standard deviation of stock return (if sig = 20%, input 0.2)
    H: barrier
    Re: rebate
    isCall: True if call option, vice versa
    isUp: True if up option, vice versa
    isOut: True if out option, vice versa
    """
    R = 1 + r 
    Q = 1 + q
    sig_T = sig * np.sqrt(T) 
    
    # Set eta and phi
    if isCall:
        phi = 1
    else: phi = -1
    if isUp:
        eta = -1
    else:
        eta = 1
    
    Q = 1 + q # Since dividend yield = 0% in the question setting
    mu = log(R/Q) - (sig**2) / 2 # Compute the mu used in the paper
    
    # Compute lambda, x, x_1, y, y_1, a, b, z used in the paper
    Lambda = 1 + mu/(sig**2)
    x = log(S/K) / (sig_T) + Lambda * sig_T
    x_1 = log(S/H) / (sig_T) + Lambda * sig_T
    y = log(H**2/(S*K)) / (sig_T) + Lambda * sig_T
    y_1 = log(H/S) / (sig_T) + Lambda * sig_T
    a = mu / (sig**2)
    b = np.sqrt(mu**2 + 2 * log(R) * (sig**2)) / (sig**2)
    z = log(H/S) / sig_T + b*sig_T
    
    # Calculate [1], [2], [3], [4], [5], [6] used for computing the value of the option
    _1 = phi*S*(Q**(-T))*norm.cdf(phi*x) - phi*K*(R**(-T))*norm.cdf(phi*x - phi*sig_T)
    _2 = phi*S*(Q**(-T))*norm.cdf(phi*x_1) - phi*K*(R**(-T))*norm.cdf(phi*x_1 - phi*sig_T)
    _3 = phi*S*(Q**(-T))*((H/S)**(2*Lambda))*norm.cdf(eta*y) - phi*K*(R**(-T))*((H/S)**(2*Lambda-2))*norm.cdf(eta*y - eta*sig_T)
    _4 = phi*S*(Q**(-T))*((H/S)**(2*Lambda))*norm.cdf(eta*y_1) - phi*K*(R**(-T))*((H/S)**(2*Lambda-2))*norm.cdf(eta*y_1 - eta*sig_T)
    _5 = Re*R**(-T)*(norm.cdf(eta*x_1-eta*sig_T)-(H/S)**(2*Lambda-2)*norm.cdf(eta*y_1-eta*sig_T))
    _6 = Re*((H/S)**(a+b)*norm.cdf(eta*z) + (H/S)**(a-b)*norm.cdf(eta*z - 2*eta*b*sig_T))
    
    # Compute the price
    if isCall:
        if isUp:
            if isOut:
                if K>=H:
                    return _6
                else:
                    return _1-_2+_3-_4+_6
            else:
                if K>=H:
                    return _1+_5
                else:
                    return _2-_3+_4+_5
        else:
            if isOut:
                if K>=H:
                    return _1-_3+_6
                else:
                    return _2-_4+_6
            else:
                if K>=H:
                    return _3+_5
                else:
                    return _1-_2+_4+_5 
    else:
        if isUp:
            if isOut:
                if K>=H:
                    return _2-_4+_6
                else:
                    return _1-_3+_6
            else:
                if K>=H:
                    return _1-_2+_4+_5
                else:
                    return _3+_5
        else:
            if isOut:
                if K>=H:
                    return _1-_2+_3-_4+_6
                else:
                    return _6
            else:
                if K>=H:
                    return _2-_3+_4+_5
                else:
                    return _1+_5
                